# working with date fields
## elasticSearch

1. The whole point of this notebook is to craft the JSON to set the date field to a 'date' type
    - instead of a generic 'str' type

In [1]:
!ls

a.json
broken mp3 tags.ipynb
cl.sh
curl example bulk load.ipynb
curl examples elastic.ipynb
dealing with DATE fields.ipynb
load elastic for fun and profit.ipynb
mp3-json.ipynb
music_bulk.json
music_success.log
play songs.ipynb
travel_music_dir_play_last_song_ten_seconds.ipynb
year:1900


In [2]:
def fixdate(in_date):
    out_date = in_date
    if in_date is None:
        out_date = '1900'
    out_date = out_date.replace('\x00', '')
    return(out_date)

In [3]:
import json

prune_dict = {}

song_list = []

# read from file and get data and song (10 records)
N=500
datafile = 'a.json'

with open(datafile) as myfile:
    head = [next(myfile) for x in range(N)]
for line in head:
    if "_index" in line:
        pass
    else:
        #print(line)
        song_dict = json.loads(line)
        if 'year' in song_dict:
            print(song_dict['year'])
            song_dict['year'] = fixdate(song_dict['year'])
            prune_dict['song'] = song_dict['song']
            prune_dict['year'] = song_dict['year']
            print(prune_dict)
            song_list.append(dict(prune_dict))
        

1981 
{'song': 'Passage', 'year': '1981'}
1981 
{'song': 'The Impending Death of the Vir', 'year': '1981'}
1981 
{'song': 'Pacific', 'year': '1981'}
1981 
{'song': 'Remedios', 'year': '1981'}
1981 
{'song': 'Hawk Circle', 'year': '1981'}
1981 
{'song': "Anne's Song", 'year': '1981'}
1981 
{'song': 'Processional', 'year': '1981'}
1981 
{'song': "The Bricklayer's Beautiful Dau", 'year': '1981'}
None
{'song': 'Track 20', 'year': '1900'}
None
{'song': 'Track 8', 'year': '1900'}
None
{'song': 'Track 14', 'year': '1900'}
None
{'song': 'Track 18', 'year': '1900'}
None
{'song': 'Track 3', 'year': '1900'}
None
{'song': 'Track 17', 'year': '1900'}
None
{'song': 'Track 11', 'year': '1900'}
None
{'song': 'Track 15', 'year': '1900'}
None
{'song': 'Track 7', 'year': '1900'}
None
{'song': 'Track 16', 'year': '1900'}
None
{'song': 'Track 4', 'year': '1900'}
None
{'song': 'Track 9', 'year': '1900'}
None
{'song': 'Track 19', 'year': '1900'}
None
{'song': 'Track 12', 'year': '1900'}
None
{'song': 'Track 

In [4]:


# empty

In [5]:
#print(prune_dict)

In [6]:
for sg in song_list:
    print(sg)

{'song': 'Passage', 'year': '1981'}
{'song': 'The Impending Death of the Vir', 'year': '1981'}
{'song': 'Pacific', 'year': '1981'}
{'song': 'Remedios', 'year': '1981'}
{'song': 'Hawk Circle', 'year': '1981'}
{'song': "Anne's Song", 'year': '1981'}
{'song': 'Processional', 'year': '1981'}
{'song': "The Bricklayer's Beautiful Dau", 'year': '1981'}
{'song': 'Track 20', 'year': '1900'}
{'song': 'Track 8', 'year': '1900'}
{'song': 'Track 14', 'year': '1900'}
{'song': 'Track 18', 'year': '1900'}
{'song': 'Track 3', 'year': '1900'}
{'song': 'Track 17', 'year': '1900'}
{'song': 'Track 11', 'year': '1900'}
{'song': 'Track 15', 'year': '1900'}
{'song': 'Track 7', 'year': '1900'}
{'song': 'Track 16', 'year': '1900'}
{'song': 'Track 4', 'year': '1900'}
{'song': 'Track 9', 'year': '1900'}
{'song': 'Track 19', 'year': '1900'}
{'song': 'Track 12', 'year': '1900'}
{'song': 'Track 13', 'year': '1900'}
{'song': 'Track 1', 'year': '1900'}
{'song': 'Track 6', 'year': '1900'}
{'song': 'Track 10', 'year': '

In [7]:
from elasticsearch import Elasticsearch
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'elastic', 'port': 9200}])
    if _es.ping():
        print('Yay Connect')
    else:
        print('Awww it could not connect!')
    return _es

# this is the meat of the notebook 
- set to type date instead of default text

In [8]:
def create_index(es_object, index_name='prune'):
    created = False
    # index settings
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "members": {
                "dynamic": "strict",
                "properties": {
                    "song": {
                        "type": "text"
                    },
                    "year": {
                        "type": "date",
                        "format": "yyyy-MM-dd||yyyy"
                    },
                }
            }
        }
    }

    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [9]:
# open es connection

es = connect_elasticsearch()

Yay Connect


In [10]:
# create the 'prune' index

is_created = create_index(es, 'prune')

Created Index


# verify it stuck as date

In [11]:
! curl -XGET http://elastic:9200/prune/_mappings |python -m json.tool

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   144  100   144    0     0   6467      0 --:--:-- --:--:-- --:--:--  6545
{
    "prune": {
        "mappings": {
            "members": {
                "dynamic": "strict",
                "properties": {
                    "song": {
                        "type": "text"
                    },
                    "year": {
                        "format": "yyyy-MM-dd||yyyy",
                        "type": "date"
                    }
                }
            }
        }
    }
}


In [12]:
def store_record(elastic_object, index_name, record):
    try:
        outcome = elastic_object.index(index=index_name, doc_type='members', body=record)
    except Exception as ex:
        print('Error in indexing data')
        print(str(ex))

In [13]:
for sg in song_list:
    elastic_json = json.dumps(sg)
    store_record(es,'prune',elastic_json)

In [14]:
# list indexes


    
!curl --header 'Content-Type: application/json' -XGET 'elastic:9200/_cat/indices?v'


health status index     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .kibana_1 zrEa-AoiRQWxuiHQOIAFZw   1   0         14            3     70.1kb         70.1kb
yellow open   music     cVysETwXQZawkhfYHcBV1w   5   1       2000            0        1mb            1mb
yellow open   mymusic   AYIZ5xWvRxSWvK8kVLH9KA   5   1          1            0      9.3kb          9.3kb
green  open   prune     PGz15rsiQXiRSXMsiQqp7Q   1   0        225            0     34.6kb         34.6kb


In [15]:
!curl -XGET elastic:9200/prune/_search?q={year:'1900'} |python -m json.tool

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1390  100  1390    0     0  51020      0 --:--:-- --:--:-- --:--:-- 51481
{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 1,
        "total": 1
    },
    "hits": {
        "hits": [
            {
                "_id": "3YXs62cBYbkZq2I2U7hX",
                "_index": "prune",
                "_score": 1.0,
                "_source": {
                    "song": "Track 20",
                    "year": "1900"
                },
                "_type": "members"
            },
            {
                "_id": "3oXs62cBYbkZq2I2U7hv",
                "_index": "prune",
                "_score": 1.0,
                "_source": {
                    "song": "Track 8",
                    "year": "1900"
                },
                "_type": "members"
            },
            {
     

In [16]:
%%bash
curl --header 'Content-Type: application/json' -XGET elastic:9200/prune/_search -d'
{
  "query": {
    "simple_query_string" : {
        "query": "1900 | 1981",
        "fields": ["year^5", "song"],
        "default_operator": "and"
    }
  }
}

' | python -m json.tool

{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 1,
        "total": 1
    },
    "hits": {
        "hits": [
            {
                "_id": "1YXs62cBYbkZq2I2Urh0",
                "_index": "prune",
                "_score": 5.0,
                "_source": {
                    "song": "Passage",
                    "year": "1981"
                },
                "_type": "members"
            },
            {
                "_id": "1oXs62cBYbkZq2I2Uria",
                "_index": "prune",
                "_score": 5.0,
                "_source": {
                    "song": "The Impending Death of the Vir",
                    "year": "1981"
                },
                "_type": "members"
            },
            {
                "_id": "14Xs62cBYbkZq2I2Uri8",
                "_index": "prune",
                "_score": 5.0,
                "_source": {
                    "song": "Pacific",
                    "year": "1981"
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1608  100  1445  100   163  45513   5134 --:--:-- --:--:-- --:--:-- 46612


In [17]:
!curl -XGET elastic:9200/prune/_search?q={year:'1900'} |python -m json.tool

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1390  100  1390    0     0  55568      0 --:--:-- --:--:-- --:--:-- 57916
{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 1,
        "total": 1
    },
    "hits": {
        "hits": [
            {
                "_id": "3YXs62cBYbkZq2I2U7hX",
                "_index": "prune",
                "_score": 1.0,
                "_source": {
                    "song": "Track 20",
                    "year": "1900"
                },
                "_type": "members"
            },
            {
                "_id": "3oXs62cBYbkZq2I2U7hv",
                "_index": "prune",
                "_score": 1.0,
                "_source": {
                    "song": "Track 8",
                    "year": "1900"
                },
                "_type": "members"
            },
            {
     

In [18]:
!curl -XGET elastic:9200/music/_search?q={year:'1981'} |python -m json.tool

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4500  100  4500    0     0  76182      0 --:--:-- --:--:-- --:--:-- 77586
{
    "_shards": {
        "failed": 0,
        "skipped": 0,
        "successful": 5,
        "total": 5
    },
    "hits": {
        "hits": [
            {
                "_id": "3",
                "_index": "music",
                "_score": 5.831089,
                "_source": {
                    "album": "Passage\u0000",
                    "artist": "William Ackerman\u0000",
                    "band": "William Ackerman\u0000",
                    "composer": "Will Ackerman\u0000",
                    "genre": "New Age\u0000",
                    "path": "/mnt/nuc/music-best/beets/EasyListening/William Ackerman/Passage/04 Pacific.mp3",
                    "publisher": "Windham Hill Records\u0000",
                    "song": "Pacific",
      